In [1]:
import pandas as pd
from glob import glob
from pathlib import Path
from ragas import evaluate
from datasets import Dataset
import os
from dotenv import load_dotenv

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
)


load_dotenv()
open_ai_auth = os.getenv("OPEN_AI")

os.environ["OPENAI_API_KEY"] = open_ai_auth


metrics = [
    faithfulness,
    answer_relevancy,
]

results_path = Path("./results/")

result_rows = []
for file in results_path.glob("*csv"):
    # read results csv
    df = pd.read_csv(file)
    
    # setup name
    setup = file.name
    
    # model name
    model = df["model"].values[0]
    
    # df to ds transformation
    df = pd.read_csv(file)
    df = df[["question", "context", "answer", "correct_answer"]]
    df.columns = ['question','contexts','answer', "ground_truth"]
    df["contexts"] = df["contexts"].apply(lambda c: [c])
    
    # convert df to ds
    ds = Dataset.from_pandas(df)
    
    # evaluate with ragas
    result = evaluate(ds, metrics=metrics)
    result["model"] = model
    result_rows.append(result)
    
pd.DataFrame(result_rows).to_csv("ragas_results.csv")
pd.DataFrame(result_rows)

Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/ragas/executor.py", line 93, in run
    results = self.loop.run_until_complete(self._aresults())
  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 649, in run_until_complete
    return future.result()
  File "/opt/conda/lib/python3.10/site-packages/ragas/executor.py", line 81, in _aresults
    raise e
  File "/opt/conda/lib/python3.10/site-packages/ragas/executor.py", line 76, in _aresults
    r = await future
  File "/opt/conda/lib/python3.10/asyncio/tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "/opt/conda/lib/python3.10/site-packages/ragas/executor.py", line 36, in sema_coro
    return await coro
  File "/opt/conda/lib/python3.10/site-packages/ragas/executor.py", line 109, in wrapped_callable_async
    return counter

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [5]:
result

NameError: name 'result' is not defined